In [ ]:
# Heatwave: Any period of three days or more when the maximum temperature each day is in the top 10 per cent of the local 15-day average between 2000 and 2020.
# Heatwave duration: Average length of heatwave event between 2000 and 2020 (Number of days).
# Heatwave severity: Average exceedance in degrees Celsius of the heatwave threshold for each event between 2000 and 2020.
# Extreme high temperature: Annual average number of days in which 35°C is exceeded between 2000 and 2020.

In [72]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
import ee
import geopandas as gpd
import pandas as pd
import geopandas as gpd
import numpy as np
import time


In [91]:
# Implements a delegation chain to the target
# credential.  Requires updates to IAM!
# private_key='/content/drive/MyDrive/Colab Notebooks/gee/private-key.json'
# service_account = 'climate-gee-project@ee-dohyungnim.iam.gserviceaccount.com'
# credentials = ee.ServiceAccountCredentials(service_account, private_key)
# ee.Initialize(credentials=credentials, project='ee-dohyungnim',opt_url='https://earthengine-highvolume.googleapis.com')


In [92]:
ee.Authenticate()
ee.Initialize(project='ee-dohyungnim',opt_url='https://earthengine-highvolume.googleapis.com')

In [93]:
# Load ERA5 Daily Temperature dataset.
era5_temp = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")

In [104]:
# Define the time range of baseline
start_base = '1960-01-01'
end_base = '1989-12-31'

#range of years for hwi caluclation
start_calc_year = 1960
end_calc_year = 2024

# Define total days and batch size
total_days = 365
batch_size = 10  #Define the number of days per batch

dailyMeansCollection = ee.ImageCollection([]);



In [105]:
grid_file = "/content/drive/MyDrive/CCRI/10degree_selected_tiles.csv"
data = pd.read_csv(grid_file)
minx = data['left']
maxy = data['top']
maxx = data['right']
miny = data['bottom']
tile_id =data["id"]
grid_count = len(minx)

In [106]:
era5_daily_temp_aoi = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")\
                                    .filterDate(start_base, end_base)\
                                    .select('temperature_2m_max')
projection = era5_daily_temp_aoi.first().projection().getInfo()
mask = era5_daily_temp_aoi.first().mask()

In [107]:
def get_90th_percentile_temp(aoi, grid_id, start_base, end_base, total_days=365, batch_size=10):
    # Load the ERA5 daily temperature dataset
    era5_daily_temp_aoi = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")\
                                    .filterDate(start_base, end_base)\
                                    .select('temperature_2m_max')

    # Define the collection to store the final results
    final_collection = ee.ImageCollection([])

    # Function to process images for a given day of the year
    def process_day(day_of_year):
        doy_filter = ee.Filter.calendarRange(day_of_year, day_of_year, 'day_of_year')
        filtered_images = era5_daily_temp_aoi.filter(doy_filter)
        daily_stack = filtered_images.toBands()
        return daily_stack.set('day_of_year', day_of_year)

    # Iterate over each batch of days within the total days range
    for start_day in range(1, total_days + 1, batch_size):
        end_day = start_day + batch_size - 1
        if end_day > total_days:
            end_day = total_days
        # Generate a list of day sequences for the batch
        days_batch = ee.List.sequence(start_day, end_day)
        # Process each day in the batch and create an ImageCollection
        batch_result = ee.ImageCollection.fromImages(days_batch.map(process_day))
        # Merge the batch results into the final collection
        final_collection = final_collection.merge(batch_result)

    daily_means_collection = final_collection
    final_collection = ee.ImageCollection([])

    def calc_rolling_90th_percentile(day_of_year):
        start_doy = ee.Number(day_of_year).subtract(7).max(1)
        end_doy = ee.Number(day_of_year).add(7).min(365)  # Adjusted for standard year
        window_collection = daily_means_collection.filter(ee.Filter.rangeContains('day_of_year', start_doy, end_doy))
        # Flatten the ImageCollection to a single Image with all bands for percentile calculation
        window_stack = window_collection.toBands()
        percentile_90_image = window_stack.reduce(ee.Reducer.percentile([90]))
        return percentile_90_image.set('day_of_year', day_of_year)

    for start_day in range(1, total_days + 1, batch_size):
        end_day = start_day + batch_size - 1
        if end_day > total_days:
            end_day = total_days  # Adjust for the last batch to not exceed the total days
        days_batch = ee.List.sequence(start_day, end_day)
        batch_result = days_batch.map(calc_rolling_90th_percentile)
        final_collection = final_collection.merge(ee.ImageCollection.fromImages(batch_result))

    rolling_90th_percentile_collection = final_collection

    return rolling_90th_percentile_collection


In [108]:
def calc_indicators(rolling_90th_percentile_collection, start_date, end_date, year):
    # Load and filter the ERA5 daily temperature dataset within the AOI
    era5_daily_target = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")\
                                  .filterDate(start_date, end_date)\
                                  .select('temperature_2m_max')


    # Function to identify extreme temperature days
    def identify_extremes(image):
        day_of_year = ee.Date(image.date()).getRelative('day', 'year').add(1)
        percentile_90_image = rolling_90th_percentile_collection.filterMetadata('day_of_year', 'equals', day_of_year).first()

        #extreme_temp = image.gt(percentile_90_image)
        extreme_temp_degree = image.gt(308.15)  # 35 degrees Celsius in Kelvin
        temp_difference = image.subtract(percentile_90_image)

        return ee.Image.cat(temp_difference.rename('temp_diff'),
                            extreme_temp_degree.rename('extreme_temp_degree'))

    # Map the function over the daily temperature collection
    extremes_collection = era5_daily_target.map(identify_extremes)
    # Sum all the temperature differences for extreme days
    temp_diff_collection = extremes_collection.select('temp_diff')
    #extreme high temperature - higher than 35C
    ext_high_temp_days = extremes_collection.select('extreme_temp_degree')
    total_high_temp_degree_days = ext_high_temp_days.sum();

    def update_streaks(img_tmp_diff, lists):
        img_flag = img_tmp_diff.gt(0)

        lists = ee.List(lists)
        streak_count_list = ee.List(lists.get(0))  # Current streak counts
        streak_total_list = ee.List(lists.get(1))  # Total streaks count
        streak_days_list = ee.List(lists.get(2))   # Total days in valid streaks
        streak_diff_list = ee.List(lists.get(3))   # Total temperature difference in valid streaks
        streak_tmp_diff_list = ee.List(lists.get(4)) # Cumulative temperature difference

        last_streak_count = ee.Image(streak_count_list.get(-1))
        total_streaks = ee.Image(streak_total_list.get(-1))
        total_days = ee.Image(streak_days_list.get(-1))
        total_diff = ee.Image(streak_diff_list.get(-1))
        total_diff_tmp = ee.Image(streak_tmp_diff_list.get(-1))

        current_flag = img_flag.neq(0)  # Continue streak if not 0
        new_streak_count = last_streak_count.add(current_flag).multiply(current_flag).float()
        new_tmp_diff = total_diff_tmp.add(img_tmp_diff).multiply(current_flag).float()

        # Check where a valid streak has potentially ended
        ended_streak = last_streak_count.gte(3).And(img_flag.eq(0))
        total_streaks = total_streaks.add(ended_streak).float()
        total_days = total_days.add(last_streak_count.multiply(ended_streak)).float()
        total_diff = total_diff.add(total_diff_tmp.multiply(ended_streak)).float()

        updated_streak_count_list = streak_count_list.add(new_streak_count)
        updated_streak_total_list = streak_total_list.add(total_streaks)
        updated_streak_days_list = streak_days_list.add(total_days)
        updated_streak_diff_list = streak_diff_list.add(total_diff)
        updated_streak_tmp_diff_list = streak_tmp_diff_list.add(new_tmp_diff)

        return ee.List([updated_streak_count_list, updated_streak_total_list, updated_streak_days_list,updated_streak_diff_list,updated_streak_tmp_diff_list])

    # Initialize with zeros
    initial_streak_count = ee.Image(0).rename('streak_count').float()
    initial_total_streaks = ee.Image(0).rename('total_streaks').float()
    initial_total_days = ee.Image(0).rename('total_days').float()
    initial_total_diff = ee.Image(0).rename('total_diff').float()
    initial_total_diff_tmp = ee.Image(0).rename('total_diff_tmp').float()

    init_lists = ee.List([
        ee.List([initial_streak_count]),
        ee.List([initial_total_streaks]),
        ee.List([initial_total_days]),
        ee.List([initial_total_diff]),
        ee.List([initial_total_diff_tmp])
    ])

    # Accumulate results
    result_lists = temp_diff_collection.iterate(update_streaks, init_lists)

    # Extract the final lists
    final_streak_count_list = ee.List(ee.List(result_lists).get(0))
    final_total_streaks_list = ee.List(ee.List(result_lists).get(1))
    final_total_days_list = ee.List(ee.List(result_lists).get(2))
    final_total_diff_list = ee.List(ee.List(result_lists).get(3))

    # Convert these lists back to ImageCollections for further analysis or visualization
    final_streak_counts = ee.ImageCollection.fromImages(final_streak_count_list)
    final_total_streaks = ee.ImageCollection.fromImages(final_total_streaks_list)
    final_total_days = ee.ImageCollection.fromImages(final_total_days_list)
    final_total_diff = ee.ImageCollection.fromImages(final_total_diff_list)

    # Calculate maximum streaks and days
    max_streaks = final_total_streaks.max()
    max_days = final_total_days.max()
    max_diff = final_total_diff.max()

    # Calculate average temperature difference for days that exceed the 90th percentile
    average_temp_diff = max_diff.divide(max_days)

    heatwave_indicators = max_streaks.float()
    heatwave_indicators = heatwave_indicators.addBands(max_days.float())
    heatwave_indicators = heatwave_indicators.addBands(average_temp_diff.float())
    heatwave_indicators = heatwave_indicators.addBands(total_high_temp_degree_days.float())
    heatwave_indicators = heatwave_indicators.rename([year +  '_hw_count', year + '_hw_days', year + '_hw_temp_diff', year + '_high_temp_degree_days'])

    return heatwave_indicators


In [109]:
def calc_indicators_val(rolling_90th_percentile_collection, start_date, end_date, year):
    # Load and filter the ERA5 daily temperature dataset within the AOI
    era5_daily_target = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")\
                                  .filterDate(start_date, end_date)\
                                  .select('temperature_2m_max')

    # Function to identify extreme temperature days
    def identify_extremes(image):
        day_of_year = ee.Date(image.date()).getRelative('day', 'year').add(1)
        percentile_90_image = rolling_90th_percentile_collection.filterMetadata('day_of_year', 'equals', day_of_year).first()

        #extreme_temp = image.gt(percentile_90_image)
        extreme_temp_degree = image.gt(308.15)  # 35 degrees Celsius in Kelvin
        temp_difference = image.subtract(percentile_90_image)

        return ee.Image.cat(temp_difference.rename('temp_diff'),
                            extreme_temp_degree.rename('extreme_temp_degree'))

    # Map the function over the daily temperature collection
    extremes_collection = era5_daily_target.map(identify_extremes)
    # Sum all the temperature differences for extreme days
    temp_diff_collection = extremes_collection.select('temp_diff')

    return temp_diff_collection


In [110]:
def export_data(final_result, aoi, grid_id, folder):
#band1: total number of heatwaves
#band2: total days of heatwaves
#band3: average tempearture difference in extreme heat days and 90 percentile th. scale factor =10
#band4: extreme heat days over 35 degree celsius

    down_args = {
    'image': final_result,
    'folder': folder,
    'fileNamePrefix': 'hwi_' + grid_id,
    'description': 'Heatwaves indicators',
    'region': aoi,
    'maxPixels': 1e13,
    'crs': projection['crs'],
    'crsTransform' : projection['transform'],
    'fileFormat': 'GeoTIFF'
    }
    task = ee.batch.Export.image.toDrive(**down_args)
    task.start()

In [111]:
# Apply the batch processing function
#for i in range(120, 121):
for i in range(grid_count):
    coordinates = np.array([minx[i], maxy[i], maxx[i], miny[i]])
    aoi = ee.Geometry.Rectangle(coordinates.tolist())
    grid_id = str(tile_id[i])

    rolling_90th_percentile_collection = get_90th_percentile_temp(aoi, grid_id, start_base, end_base, total_days, batch_size)

    # Iterate by decades
    for start_year in range(1960, 2024, 10):
        end_year = min(start_year + 9, 2023)
        folder = 'hwi_' + str(start_year) + 's'
        final_result = mask.float()  # init land mask as the first layer
        for y in range(start_year, end_year + 1):
            year = str(y)
            start_date = year + '-01-01'
            end_date = year + '-12-31'
            final_result = final_result.addBands(calc_indicators(rolling_90th_percentile_collection, start_date, end_date, year))

        export_data(final_result, aoi, grid_id,folder)

In [102]:
def cancel_all_tasks():
    tasks = ee.batch.Task.list()
    for task in tasks:
        if task.active():  # Check if the task is active
            print(f"Cancelling task {task.id} ({task.config['description']})")
            task.cancel()

In [103]:
# Call the function to cancel all tasks
cancel_all_tasks()

Cancelling task 5HW5VMSOCX2AMID4YGWFP5TE (Heatwaves indicators)
Cancelling task EK5AT65ATTAJ4PIAB5ZZUMPP (Heatwaves indicators)
Cancelling task WUKZ6KOUKCNM53SWVGKEF3ZH (Heatwaves indicators)
Cancelling task 3D76YIMTMPEQL5ULQ22DAYM7 (Heatwaves indicators)
Cancelling task 5ZQLBZJFSQWSFTJW5FZW6WVP (Heatwaves indicators)


EEException: Operation "projects/ee-dohyungnim/operations/KRXJSRYS2UQMHZ7INKQVCC27" not found.